<a href="https://colab.research.google.com/github/kevoniano/AI-stuff/blob/master/Frozen_Lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Gym y Q-learning para resolver un problema usando un MDP

[AI Gym](http://gym.openai.com/docs/) es una librería que contiene ambientes para probar algoritmos de IA. 

AI Gym es útil para no tener que desarrollar ambientes de prueba y para poder comparar resultados entre experimentos directamente. 

# Instalar dependencias



In [0]:
import gym
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

# Crear un ambiente
Vamos a jugar con [Frozen lake](https://gym.openai.com/envs/FrozenLake-v0/). 

El invierno esta aqui. Tú y tus amigos están jugando con un frisbee cerca de un lago cuando uno de ellos se aloca y avienta el frisbee al centro del lago. 

El agua está mayormente congelada, pero hay algunos agujeros donde el hielo se ha derretido. Si entras en uno de esos agujeros, caerás en el agua helada (RIP: apareces en las noticias). Pero vale la pena arriesgarse porque hay una escasez internacional de frisbees, por lo que es absolutamente imperativo que vayas al lago y recuperes el disco. **Sin embargo, el hielo es resbaladizo, por lo que no siempre te moverás en la dirección que deseas**. 

El lago se muestra como un texto. En **S** empiezas, **F** es el hielo, **H** es el hoyo mortal y **G** es a donde tienes que llegar. 


In [2]:
# Instanciamos el ambiente
env = gym.make('FrozenLake-v0')
# Lo inicializamos
obs= env.reset()
# Mostramos el estado actual
env.render()


SFFF
FHFH
FFFH
HFFG


# Muevete en el lago

Juega en el lago. Estas son las acciones:

- LEFT = 0
- DOWN = 1
- RIGHT = 2
- UP = 3

In [3]:
# Muevete a la izquierda y luego arriba
obs= env.reset()
# Ejecuta la accion 0
env.step(0)
env.render()
# Ejecuta la accion 3
env.step(3)
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG


In [4]:
# Esta secuencia de pasos deberia llevarte a la celda G. O no?
obs= env.reset()
actions = [1, 1, 2, 1, 2, 2]
for a in actions: 
  env.step(a)
  env.render()

  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


In [6]:
# O usa el teclado para moverte, tienes  20 intentos.
# LEFT = 0
# DOWN = 1
# RIGHT = 2
# UP = 3
limit_steps = 20
obs= env.reset()
env.render()
for s in range(limit_steps):
  user_action = int(input("Accion: "))
  env.step(user_action)
  env.render()
print()
print("No pudiste ja ja")


SFFF
FHFH
FFFH
HFFG
Accion: 1
  (Down)
SFFF
FHFH
FFFH
HFFG
Accion: 3
  (Up)
SFFF
FHFH
FFFH
HFFG
Accion: 0
  (Left)
SFFF
FHFH
FFFH
HFFG
Accion: 1
  (Down)
SFFF
FHFH
FFFH
HFFG
Accion: 2
  (Right)
SFFF
FHFH
FFFH
HFFG


KeyboardInterrupt: ignored

# Ahora usemos RL

In [8]:
env = gym.make('FrozenLake-v0')
env.reset()

# Usamos metodos especiales para acceder al numero de acciones del ambiente
NUM_ACTIONS = env.action_space.n
NUM_STATES = env.observation_space.n

# Creamos la tabla Q
Q = np.zeros([NUM_STATES, NUM_ACTIONS]) 

gamma = 0.95  # factor de descuento
alpha = 0.01  # tasa de aprendizaje
epsilon = 0.1  # para epsilon greedy

print(NUM_ACTIONS)
print(NUM_STATES)

4
16


In [13]:
# ahora vamos a probar la tabla Q entrenada (es probable que te caigas todavía!)
rew_tot=0.
obs= env.reset()
done=False
while done != True: 
    # explotar
    action = np.argmax(Q[obs])
    obs, rew, done, info = env.step(action) 
    print(obs, rew, done, info)
    rew_tot += rew
    env.render()

print("Recompensa:", rew_tot)  
if(rew_tot == 1):
  print("Encontraste tu frisbee :)")
else:
  print("Saliste en las noticias :(")

4 0.0 False {'prob': 0.3333333333333333}
  (Left)
SFFF
FHFH
FFFH
HFFG
8 0.0 False {'prob': 0.3333333333333333}
  (Left)
SFFF
FHFH
FFFH
HFFG
9 0.0 False {'prob': 0.3333333333333333}
  (Up)
SFFF
FHFH
FFFH
HFFG
13 0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
14 0.0 False {'prob': 0.3333333333333333}
  (Right)
SFFF
FHFH
FFFH
HFFG
14 0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
14 0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
15 1.0 True {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
Recompensa: 1.0
Encontraste tu frisbee :)


In [9]:
# Vamos a entrenar por 500,000 episodios
for episode in range(1,500001):
    done = False
    obs = env.reset()
    while done != True:
        if np.random.rand(1) < epsilon:
            # exploramos con probabilidad epsilon (epsilon greedy)
            action = env.action_space.sample()
        else:
            # explotar
            action = np.argmax(Q[obs])
            
        # ejecutar una accion
        obs2, rew, done, info = env.step(action) 
        
        # actualizar tabla Q
        # aqui sucede el aprendizaje -------------------------------------------
        Q[obs,action] += alpha * (rew + gamma * np.max(Q[obs2]) - Q[obs,action]) 
        # ----------------------------------------------------------------------
        
        obs = obs2   
        
    if episode % 5000 == 0:
        # reportar cada 5,000 pasos
        # probar cada 100 episodios para calcular el puntaje promedio y verificar si esta resuelto
        rew_average = 0.
        for i in range(100):
            obs= env.reset()
            done=False
            while done != True: 
                # explotar
                action = np.argmax(Q[obs])
                obs, rew, done, info = env.step(action) 
                rew_average += rew
        rew_average=rew_average/100
        print('Episodio: {}. Recompensa promedio: {}'.format(episode,rew_average))
        
        if rew_average > 0.8:
            # FrozenLake-v0 defines "solving" as getting average reward of 0.78 over 100 consecutive trials.
            # FrozenLake-v0 define "resolver" como tener un promedio de 0.78 en 100 intentos
            # Somos exigentes y probamos con 0.8
            print("Problema resuelto: encontraste tu frisbee. ")
            break

Episodio: 5000. Recompensa promedio: 0.0
Episodio: 10000. Recompensa promedio: 0.0
Episodio: 15000. Recompensa promedio: 0.0
Episodio: 20000. Recompensa promedio: 0.05
Episodio: 25000. Recompensa promedio: 0.13
Episodio: 30000. Recompensa promedio: 0.71
Episodio: 35000. Recompensa promedio: 0.76
Episodio: 40000. Recompensa promedio: 0.69
Episodio: 45000. Recompensa promedio: 0.68
Episodio: 50000. Recompensa promedio: 0.78
Episodio: 55000. Recompensa promedio: 0.75
Episodio: 60000. Recompensa promedio: 0.77
Episodio: 65000. Recompensa promedio: 0.71
Episodio: 70000. Recompensa promedio: 0.76
Episodio: 75000. Recompensa promedio: 0.67
Episodio: 80000. Recompensa promedio: 0.77
Episodio: 85000. Recompensa promedio: 0.7
Episodio: 90000. Recompensa promedio: 0.8
Episodio: 95000. Recompensa promedio: 0.67
Episodio: 100000. Recompensa promedio: 0.64
Episodio: 105000. Recompensa promedio: 0.75
Episodio: 110000. Recompensa promedio: 0.81
Problema resuelto: encontraste tu frisbee. 


In [10]:
print("Esta es la tabla Q")
print(Q)

Esta es la tabla Q
[[0.17935732 0.1687829  0.16394224 0.16133913]
 [0.10985574 0.09903054 0.10143319 0.15055548]
 [0.14640311 0.14138566 0.14198603 0.13638419]
 [0.04617387 0.04162181 0.05220083 0.12923519]
 [0.20723859 0.15536446 0.12639106 0.11260476]
 [0.         0.         0.         0.        ]
 [0.15575635 0.10454368 0.16095946 0.04800113]
 [0.         0.         0.         0.        ]
 [0.14383266 0.20299543 0.1748917  0.26229952]
 [0.26182872 0.36090822 0.28616555 0.20983711]
 [0.3869271  0.35541888 0.26343635 0.16217693]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.28022672 0.35642439 0.4952045  0.33908445]
 [0.50680717 0.70378203 0.68389865 0.61548364]
 [0.         0.         0.         0.        ]]
